<a href="https://colab.research.google.com/github/Maurickson/projetoIntegrador/blob/main/Q5_Hackathon_Uniafcisa_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instituto: Unifacisa
# Aluno: Maurickson Xavier Braga - Data: 27/03
# Q5 - Diagnóstico de Doenças Cardíacas.

**Questão 5 (Intermediário) - Diagnóstico de Doenças Cardíacas**

 Você recebeu um dataset contendo informações médicas de pacientes, como idade, pressão arterial, colesterol e frequência cardíaca. Seu objetivo é construir um modelo de classificação para
prever se um paciente tem ou não uma doença cardíaca.


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

# Para reprodutibilidade
np.random.seed(42)

# Número de amostras
n = 303

# Gerando dados sintéticos
df = pd.DataFrame({
    'age': np.random.randint(29, 77, n),
    'sex': np.random.randint(0, 2, n),
    'cp': np.random.randint(0, 4, n),  # tipo de dor no peito
    'trestbps': np.random.randint(94, 200, n),  # pressão arterial
    'chol': np.random.randint(126, 564, n),  # colesterol
    'fbs': np.random.randint(0, 2, n),  # açúcar no sangue em jejum
    'restecg': np.random.randint(0, 2, n),  # resultados do eletrocardiograma
    'thalach': np.random.randint(71, 202, n),  # frequência cardíaca máxima
    'exang': np.random.randint(0, 2, n),  # angina induzida por exercício
    'oldpeak': np.round(np.random.uniform(0.0, 6.2, n), 1),  # depressão ST
    'slope': np.random.randint(0, 3, n),
    'ca': np.random.randint(0, 4, n),  # número de vasos coloridos
    'thal': np.random.randint(0, 3, n),  # tipo de thalassemia
    'target': np.random.randint(0, 2, n)  # variável alvo
})


df.head()




,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,67,1,2,126,458,1,0,144,0,2.3,2,3,1,1
1,57,0,0,158,384,0,1,133,0,6.2,0,2,1,0
2,43,0,3,111,286,0,0,130,0,2.8,1,0,2,0
3,71,1,2,189,515,1,1,149,0,2.1,1,0,2,1
4,36,0,0,142,303,0,0,107,1,3.6,1,2,1,1


In [7]:
# Verificar valores ausentes
print("Valores ausentes por coluna:")
print(df.isnull().sum())

# Separando features (X) e target (y)
X = df.drop('target', axis=1)
y = df['target']

# Normalizando os dados (padronização)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão treino/teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)


Valores ausentes por coluna:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [9]:
# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

# SVM (com probabilidade ativada para curva ROC)
svm_model = SVC(probability=True, random_state=42)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)


In [12]:
# Função de avaliação
def avaliar_modelo(nome, y_true, y_pred, y_proba):
    print(f"\n=== {nome} ===")
    print(classification_report(y_true, y_pred, digits=3))
    auc = roc_auc_score(y_true, y_proba)
    print(f"ROC AUC: {auc:.4f}")
    return auc

# Avaliando Random Forest
rf_auc = avaliar_modelo("Random Forest", y_test, rf_preds, rf_model.predict_proba(X_test)[:, 1])

# Avaliando SVM
svm_auc = avaliar_modelo("SVM", y_test, svm_preds, svm_model.predict_proba(X_test)[:, 1])



=== Random Forest ===
              precision    recall  f1-score   support

           0      0.607     0.548     0.576        31
           1      0.576     0.633     0.603        30

    accuracy                          0.590        61
   macro avg      0.591     0.591     0.590        61
weighted avg      0.592     0.590     0.590        61

ROC AUC: 0.6054

=== SVM ===
              precision    recall  f1-score   support

           0      0.625     0.484     0.545        31
           1      0.568     0.700     0.627        30

    accuracy                          0.590        61
   macro avg      0.596     0.592     0.586        61
weighted avg      0.597     0.590     0.585        61

ROC AUC: 0.6392


**Pergunta: Qual modelo apresentou melhor desempenho? Quais variáveis mais impactaram na previsão?**

- O modelo SVM apresentou um desempenho ligeiramente melhor que o Random Forest. Porque indicou uma melhor capacidade de distinguir os pacientes com e sem doença cardíaca. Porque o recall do SVM para a classe 1 (pacientes com doença cardíaca) foi superior, com 70% contra 63.3% do Random Forest.

- Por outro lado, o Random Forest teve uma precisão ligeiramente melhor para a classe 0 (sem doença), com 60.7% contra 62.5% do SVM.

- Em relação às variáveis mais impactantes, o modelo Random Forest, que oferece uma avaliação direta de importância das features, pode nos dizer quais atributos (como idade, pressão arterial, colesterol, etc.) mais influenciam na previsão da doença cardíaca.